# ノート

磁場は長岡係数 $K_n$ による補正が必要である
$$
    H = K_N nI
$$

# コード

## ライブラリのインポート

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
import os

## 測定値の取り込み

In [2]:
def get_length(N):
    if(N == 50):
        L = 7.478
    elif(N == 75):
        L = 7.866
    if(N == 100):
        L = 7.238
    if(N == 150):
        L = 8.275
    return L

def get_Nagaoka_const(N):
    Kn = 1
    # if(N == 50):
    #     Kn = 7.478
    # elif(N == 75):
    #     Kn = 7.866
    # if(N == 100):
    #     Kn = 7.238
    # if(N == 150):
    #     Kn = 8.275
    return Kn

def calc_H(N, V):
    R = 0.1 # 抵抗
    Kn = get_Nagaoka_const(N)
    L = get_length(N)

    H = Kn * N * V / L / R # 単位 A/cm

    return H/1000 # 単位 mA/cm


Ns = np.array([150, 150, 150, 150,
        100, 100, 100, 100,
        75,  75, 75, 75,
        50, 50, 50, 50,
        150, 150, 150, 150, 150,
        150, 150, 150, 150, 150])

Vs = np.array([400, 300, 200, 100,
            400, 300, 200, 100,
            400, 300, 200, 100,
            400, 300, 200, 100,
            400, 300, 200, 100, 100,
            400, 300, 200, 100, 100])

Time = np.linspace(-0.00005, 00.000449800000, 2500) * 10 ** 6


nfile = 26
raw_data = np.zeros((nfile, 2, 2500))

Intensity = np.zeros((nfile, 2500))
H_field = np.zeros((nfile, 2500))

windowsize = 100
Intensity_smooth = np.zeros((nfile, 2500))
H_field_smooth = np.zeros((nfile, 2500))

for i in range(1, nfile+1):

    # 光の強さ
    ch1 = pd.read_csv(f"12_05/ALL{i:04}/F{i:04}CH1.CSV", header=None, usecols = [4]).to_numpy()[:,0]

    # 磁場の強さ 16 番目のデータだけ破損している
    if(i != 16):
        ch2 = pd.read_csv(f"12_05/ALL{i:04}/F{i:04}CH2.CSV", header=None, usecols = [4]).to_numpy()[:,0]
    else :
        ch2 = pd.read_csv(f"12_05/ALL{i-1:04}/F{i-1:04}CH2.CSV", header=None, usecols = [4]).to_numpy()[:,0]/2

    raw_data[i-1][0] = ch1
    raw_data[i-1][1] = ch2

    Intensity[i-1] = (ch1-np.min(ch1))/(np.max(ch1)-np.min(ch1))
    H_field[i-1] = calc_H(Ns[i-1], ch2)

    Intensity_smooth[i-1] = savgol_filter(Intensity[i-1], window_length = windowsize, polyorder = 5)
    H_field_smooth[i-1] = savgol_filter(H_field[i-1], window_length = windowsize, polyorder = 5)
del i

## 出力

### X-T グラフ

In [8]:
font_size = 18

def plot_xt_graph(index):
    f = plt.figure()

    ax1 = f.add_subplot(111)
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    plt.rcParams['font.size'] = font_size
    plt.rcParams['font.family'] ='Times New Roman'
    plt.rcParams['mathtext.fontset'] = 'stix'

    ax1.set_xlim(-50,450)
    ax1.set_ylim(0,1.0)

    ax1.set_xlabel("Time ($\mathrm{\mu s}$)")
    ax1.set_ylabel("$I$ (arb. unit)")

    ax1.plot(Time, Intensity_smooth[index], c = "tab:blue", label="$I$")
    ax1.scatter(Time, Intensity[index], s = 0.1, c = "tab:blue")

    ax2 = ax1.twinx()
    ax2.set_ylabel("$H$ (kA/cm)")

    ax2.plot(Time, H_field_smooth[index], c = "tab:orange", label = "$H$")
    ax2.scatter(Time, H_field[index], s = 0.1, c = "tab:orange")

    plt.title(f"$N$ = {Ns[index]}, $V$ = {Vs[index]}")

    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2, loc='upper right')

    path = f"../image/xt/"
    if not os.path.isdir(path):
        os.makedirs(path)

    plt.savefig(f"../image/xt/{index+1:02}.png",  bbox_inches='tight')
    plt.close()


for i in range(nfile):
    plot_xt_graph(i)

### X-Y グラフ

In [13]:
def get_shift(N):
    if(N == 50):
        shift = 13
    elif(N == 75):
        shift = 14
    if(N == 100):
        shift = 19
    if(N == 150):
        shift = 24
    return shift


def plot_xy_graph(index):
    f = plt.figure()

    ax1 = f.add_subplot(111)
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    plt.rcParams['font.size'] = font_size
    plt.rcParams['font.family'] ='Times New Roman'
    plt.rcParams['mathtext.fontset'] = 'stix'

    ax1.set_xlim(0,8)
    ax1.set_ylim(0,1.0)

    ax1.set_xlabel("$H$ (kA/cm)")
    ax1.set_ylabel("Intensity (arb. unit)")

    ax1.plot(H_field_smooth[index], Intensity_smooth[index], c = "tab:blue", label = "no fixed")
    ax1.scatter(H_field[index], Intensity[index], s = 0.1, c = "tab:blue")

    shift = get_shift(Ns[index])
    ax1.plot(H_field_smooth[index][:2500-shift], Intensity_smooth[index][shift:], c = "tab:orange", label = "fixed")
    ax1.scatter(H_field[index][:2500-shift], Intensity[index][shift:], s = 0.1, c = "tab:orange")

    plt.title(f"$N$ = {Ns[index]}, $V$ = {Vs[index]}, shift = {shift}")

    plt.legend(loc = "upper right")

    path = f"../image/xy/"
    if not os.path.isdir(path):
        os.makedirs(path)

    plt.savefig(f"../image/xy/{index+1:02}.png",  bbox_inches='tight')
    plt.close()

for i in range(nfile):
    plot_xy_graph(i)

#### 磁化の応答の遅れを見る

In [6]:
def plot_xy_graph_test(index, shift = 0):
    font_size = 18
    f = plt.figure()

    ax1 = f.add_subplot(111)
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    plt.rcParams['font.size'] = font_size
    plt.rcParams['font.family'] ='Times New Roman'
    plt.rcParams['mathtext.fontset'] = 'stix'

    ax1.set_xlim(0,8)
    ax1.set_ylim(0,1.0)

    ax1.set_xlabel("$H$ (mA/cm)")
    ax1.set_ylabel("Intensity (arb. unit)")

    ax1.plot(H_field_smooth[index][:2500-shift], Intensity_smooth[index][shift:], c = "tab:blue")
    ax1.scatter(H_field[index][:2500-shift], Intensity[index][shift:], s = 0.1, c = "tab:blue")

    plt.title(f"$N$ = {Ns[index]}, $V$ = {Vs[index]}, shift = {shift}")

    path = f"../image/xy_test/{index+1:02}/"
    if not os.path.isdir(path):
        os.makedirs(path)

    plt.savefig(f"../image/xy_test/{index+1:02}/shift{shift:02}.png",  bbox_inches='tight')
    plt.close()


# for i in range(nfile):
#     for j in range(25):
#         plot_xy_graph_test(i, j)
# del i, j